In [164]:
import pandapower as pp
import pandapower.plotting as pplot
import pandas as pd
import numpy as np

In [165]:
Distrib_V_nom_kV = 400e-3 #distribution voltage
Trans_V_nom_kV = 10 #transmission/generation voltage
VA_nom_MVA = 3333e-6
VA_nom_MVASch = 5555e-3 #school load to be verified

distribution_length = 0.06 #between 2 homes
transmission_length = 0.06
transmission_district_length = 0.06
p_demand = 3000e-6 #power demand per home
q_demand = 1452.9e-6 #reactive power per home
p_demandsch = 5000e-6 #school power demand
q_dmandsch = 2412e-6 #school reactive power

Ro = 2.65e-8  #Aluminium resistivity
A = 100e-6    #Area of the cable
#L = 100       #Length
R_per_km = Ro/(A*1000) #Resistance
x_per_km = 0

In [166]:
net = pp.create_empty_network()

In [167]:
#defining the districts, strings, and houses
districts = 5
#houses_per_string = 15
strings_per_district = 2
terminals_per_string = 5
#house_bus = [None] * houses_per_string
house_bus_terminal = [None] * terminals_per_string
distribution = [None]*districts
transmission = [None]*districts

In [168]:
central_generation = pp.create_bus(net, vn_kv = Trans_V_nom_kV, name = "Generation Plant")
central_transmission = pp.create_bus(net, vn_kv = Trans_V_nom_kV, name = "Central Transmission Substation")
pp.create_ext_grid(net, bus=central_transmission, vm_pu=1, name="Swing bus")

0

In [169]:
for district in range(districts):
    transmission[district] = pp.create_bus(net, vn_kv = Trans_V_nom_kV, \
                                           name = "Transmission Substation - District %s" %district)
    distribution[district] = pp.create_bus(net, vn_kv = Distrib_V_nom_kV, \
                                           name = "Distribution Substation - District %s" %district)
    pp.create_transformer(net, transmission[district], distribution[district], \
                          std_type = "0.4 MVA 10/0.4 kV", name="TD tx %s" %district)
    if(district == 0):
        pp.create_line_from_parameters(net,from_bus= central_transmission, to_bus= transmission[district], \
                                                length_km=transmission_length, r_ohm_per_km = R_per_km,\
                                                x_ohm_per_km = x_per_km,\
                                                c_nf_per_km = 1e-9, max_i_ka=1000,\
                                                name = "Central transmission to transmission %s" %district)
    else:
        pp.create_line_from_parameters(net,from_bus=transmission[district-1],\
                                               to_bus=transmission[district],length_km=transmission_length,\
                                               r_ohm_per_km = R_per_km, x_ohm_per_km = x_per_km,\
                                               c_nf_per_km = 1e-9, max_i_ka=1000,\
                                               name = "Transmission %s to Transmission %s" %(district-1, district))
    for string in range(strings_per_district):
        for terminal in range(terminals_per_string):
            house_bus_terminal[terminal] = pp.create_bus(net, vn_kv = Distrib_V_nom_kV, \
                                   name = "District %s string %s terminal%s" %(district, string, terminal))
            pp.create_load(net, bus= house_bus_terminal[terminal], p_mw=p_demand, q_mvar=q_demand, \
                  name= "district%s  string%s  terminal %s" %(district,string,terminal))
            if (terminal == 0):
                pp.create_line_from_parameters(net,from_bus= distribution[district], to_bus= house_bus_terminal[terminal], \
                                        length_km=distribution_length, r_ohm_per_km = R_per_km,\
                                       x_ohm_per_km = x_per_km,c_nf_per_km = 1e-9, max_i_ka=1000,\
                                    name = "Distribution %s to string %s to house terminal %s" %(district,string,terminal))
            else:
                pp.create_line_from_parameters(net,from_bus=house_bus_terminal[terminal-1] ,\
                                               to_bus = house_bus_terminal[terminal],length_km=distribution_length,\
                                               r_ohm_per_km = R_per_km, x_ohm_per_km = x_per_km,\
                                               c_nf_per_km = 1e-9, max_i_ka=1000,\
                                               name = "District %s string %s terminal %s to Terminal %s" \
                                           %(district,string,terminal-1, terminal))


In [170]:
net.bus

,name,vn_kv,type,zone,in_service
0,Generation Plant,10.0,b,None,True
1,Central Transmission Substation,10.0,b,None,True
2,Transmission Substation - District 0,10.0,b,None,True
3,Distribution Substation - District 0,0.4,b,None,True
4,District 0 string 0 terminal0,0.4,b,None,True
...,...,...,...,...,...
57,District 4 string 1 terminal0,0.4,b,None,True
58,District 4 string 1 terminal1,0.4,b,None,True
59,District 4 string 1 terminal2,0.4,b,None,True
60,District 4 string 1 terminal3,0.4,b,None,True


In [171]:
net.line

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service
0,Central transmission to transmission 0,None,1,2,0.06,2.650000e-07,0.0,1.000000e-09,0.0,1000.0,1.0,1,None,True
1,Distribution 0 to string 0 to house terminal 0,None,3,4,0.06,2.650000e-07,0.0,1.000000e-09,0.0,1000.0,1.0,1,None,True
2,District 0 string 0 terminal 0 to Terminal 1,None,4,5,0.06,2.650000e-07,0.0,1.000000e-09,0.0,1000.0,1.0,1,None,True
3,District 0 string 0 terminal 1 to Terminal 2,None,5,6,0.06,2.650000e-07,0.0,1.000000e-09,0.0,1000.0,1.0,1,None,True
4,District 0 string 0 terminal 2 to Terminal 3,None,6,7,0.06,2.650000e-07,0.0,1.000000e-09,0.0,1000.0,1.0,1,None,True
5,District 0 string 0 terminal 3 to Terminal 4,None,7,8,0.06,2.650000e-07,0.0,1.000000e-09,0.0,1000.0,1.0,1,None,True
6,Distribution 0 to string 1 to house terminal 0,None,3,9,0.06,2.650000e-07,0.0,1.000000e-09,0.0,1000.0,1.0,1,None,True
7,District 0 string 1 terminal 0 to Terminal 1,None,9,10,0.06,2.650000e-07,0.0,1.000000e-09,0.0,1000.0,1.0,1,None,True
8,District 0 string 1 terminal 1 to Terminal 2,None,10,11,0.06,2.650000e-07,0.0,1.000000e-09,0.0,1000.0,1.0,1,None,True
9,District 0 string 1 terminal 2 to Terminal 3,None,11,12,0.06,2.650000e-07,0.0,1.000000e-09,0.0,1000.0,1.0,1,None,True


In [172]:
net.load

,name,bus,p_mw,q_mvar,const_z_percent,const_i_percent,sn_mva,scaling,in_service,type
0,district0 string0 terminal 0,4,0.003,0.001453,0.0,0.0,NaN,1.0,True,wye
1,district0 string0 terminal 1,5,0.003,0.001453,0.0,0.0,NaN,1.0,True,wye
2,district0 string0 terminal 2,6,0.003,0.001453,0.0,0.0,NaN,1.0,True,wye
3,district0 string0 terminal 3,7,0.003,0.001453,0.0,0.0,NaN,1.0,True,wye
4,district0 string0 terminal 4,8,0.003,0.001453,0.0,0.0,NaN,1.0,True,wye
5,district0 string1 terminal 0,9,0.003,0.001453,0.0,0.0,NaN,1.0,True,wye
6,district0 string1 terminal 1,10,0.003,0.001453,0.0,0.0,NaN,1.0,True,wye
7,district0 string1 terminal 2,11,0.003,0.001453,0.0,0.0,NaN,1.0,True,wye
8,district0 string1 terminal 3,12,0.003,0.001453,0.0,0.0,NaN,1.0,True,wye
9,district0 string1 terminal 4,13,0.003,0.001453,0.0,0.0,NaN,1.0,True,wye


In [173]:
net.trafo

,name,std_type,hv_bus,lv_bus,sn_mva,vn_hv_kv,vn_lv_kv,vk_percent,vkr_percent,pfe_kw,...,tap_neutral,tap_min,tap_max,tap_step_percent,tap_step_degree,tap_pos,tap_phase_shifter,parallel,df,in_service
0,TD tx 0,0.4 MVA 10/0.4 kV,2,3,0.4,10.0,0.4,4.0,1.325,0.95,...,0,-2,2,2.5,0.0,0,False,1,1.0,True
1,TD tx 1,0.4 MVA 10/0.4 kV,14,15,0.4,10.0,0.4,4.0,1.325,0.95,...,0,-2,2,2.5,0.0,0,False,1,1.0,True
2,TD tx 2,0.4 MVA 10/0.4 kV,26,27,0.4,10.0,0.4,4.0,1.325,0.95,...,0,-2,2,2.5,0.0,0,False,1,1.0,True
3,TD tx 3,0.4 MVA 10/0.4 kV,38,39,0.4,10.0,0.4,4.0,1.325,0.95,...,0,-2,2,2.5,0.0,0,False,1,1.0,True
4,TD tx 4,0.4 MVA 10/0.4 kV,50,51,0.4,10.0,0.4,4.0,1.325,0.95,...,0,-2,2,2.5,0.0,0,False,1,1.0,True


In [174]:
pp.runpp(net, algorithm='nr');

In [175]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar
0,NaN,NaN,0.000000,0.000000
1,1.000000,0.000000e+00,-0.154928,-0.073185
2,1.000000,6.667208e-10,0.000000,0.000000
3,0.997611,-1.374965e-01,0.000000,0.000000
4,0.997611,-1.374965e-01,0.003000,0.001453
...,...,...,...,...
57,0.997611,-1.374965e-01,0.003000,0.001453
58,0.997611,-1.374964e-01,0.003000,0.001453
59,0.997611,-1.374964e-01,0.003000,0.001453
60,0.997611,-1.374964e-01,0.003000,0.001453


In [176]:
net.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,0.154925,0.073185,-0.154925,-0.073185,4.667933e-12,-1.873501e-15,0.009892,0.009892,0.009892,1.000000,0.000000e+00,1.000000,6.667208e-10,0.000989
1,0.015000,0.007265,-0.015000,-0.007265,2.773604e-11,0.000000e+00,0.024114,0.024114,0.024114,0.997611,-1.374965e-01,0.997611,-1.374965e-01,0.002411
2,0.012000,0.005812,-0.012000,-0.005812,1.775107e-11,0.000000e+00,0.019291,0.019291,0.019291,0.997611,-1.374965e-01,0.997611,-1.374964e-01,0.001929
3,0.009000,0.004359,-0.009000,-0.004359,9.984973e-12,-8.673617e-19,0.014468,0.014468,0.014468,0.997611,-1.374964e-01,0.997611,-1.374964e-01,0.001447
4,0.006000,0.002906,-0.006000,-0.002906,4.437770e-12,0.000000e+00,0.009645,0.009645,0.009645,0.997611,-1.374964e-01,0.997611,-1.374964e-01,0.000965
5,0.003000,0.001453,-0.003000,-0.001453,1.109441e-12,0.000000e+00,0.004823,0.004823,0.004823,0.997611,-1.374964e-01,0.997611,-1.374964e-01,0.000482
6,0.015000,0.007265,-0.015000,-0.007265,2.773604e-11,0.000000e+00,0.024114,0.024114,0.024114,0.997611,-1.374965e-01,0.997611,-1.374965e-01,0.002411
7,0.012000,0.005812,-0.012000,-0.005812,1.775107e-11,0.000000e+00,0.019291,0.019291,0.019291,0.997611,-1.374965e-01,0.997611,-1.374964e-01,0.001929
8,0.009000,0.004359,-0.009000,-0.004359,9.984973e-12,-8.673617e-19,0.014468,0.014468,0.014468,0.997611,-1.374964e-01,0.997611,-1.374964e-01,0.001447
9,0.006000,0.002906,-0.006000,-0.002906,4.437770e-12,0.000000e+00,0.009645,0.009645,0.009645,0.997611,-1.374964e-01,0.997611,-1.374964e-01,0.000965


In [177]:
transmission_school = pp.create_bus(net, vn_kv = Trans_V_nom_kV, \
                                           name = "School Transmission Substation")
distribution_school = pp.create_bus(net, vn_kv = Distrib_V_nom_kV, \
                                       name = "School Distribution Substation")
pp.create_transformer(net, transmission_school, distribution_school, \
                          std_type = "0.4 MVA 10/0.4 kV", name ="school transformer")
pp.create_line_from_parameters(net,from_bus= central_generation, to_bus= central_transmission, \
                                                length_km=transmission_length, r_ohm_per_km = R_per_km,\
                                       x_ohm_per_km = x_per_km,\
                                                c_nf_per_km = 1e-9, max_i_ka=1000,\
                                                name = "Central generation to transmission")
School_bus = pp.create_bus(net, vn_kv = Distrib_V_nom_kV, \
                                        name = "School bus")
pp.create_line_from_parameters(net,from_bus=distribution_school ,\
                                               to_bus = School_bus,length_km=transmission_length,\
                                               r_ohm_per_km = R_per_km, x_ohm_per_km = x_per_km,\
                                               c_nf_per_km = 1e-9, max_i_ka=1000,\
                                               name = "School distribution to school")
                

pp.create_load(net, bus= School_bus, p_mw=p_demandsch, q_mvar=q_dmandsch, \
                           name="school load")

pp.create_line_from_parameters(net,from_bus=transmission[4],\
                                               to_bus= transmission_school,length_km=transmission_length,\
                                               r_ohm_per_km = R_per_km, x_ohm_per_km = x_per_km,\
                                               c_nf_per_km = 1e-9, max_i_ka=1000,\
                                               name = "Transmission %s to school transmission" %(district))

57

In [178]:
pp.runpp(net, algorithm='nr');

In [179]:
net.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,0.160875,0.075601,-0.160875,-7.560076e-02,5.023842e-12,-1.873501e-15,0.010263,1.026261e-02,1.026261e-02,1.000000,0.000000e+00,1.000000,6.887251e-10,1.026261e-03
1,0.015000,0.007264,-0.015000,-7.264500e-03,2.773605e-11,0.000000e+00,0.024114,2.411366e-02,2.411366e-02,0.997611,-1.374965e-01,0.997611,-1.374965e-01,2.411366e-03
2,0.012000,0.005812,-0.012000,-5.811600e-03,1.775106e-11,0.000000e+00,0.019291,1.929092e-02,1.929092e-02,0.997611,-1.374965e-01,0.997611,-1.374964e-01,1.929092e-03
3,0.009000,0.004359,-0.009000,-4.358700e-03,9.984976e-12,8.673617e-19,0.014468,1.446819e-02,1.446819e-02,0.997611,-1.374964e-01,0.997611,-1.374964e-01,1.446819e-03
4,0.006000,0.002906,-0.006000,-2.905800e-03,4.437766e-12,0.000000e+00,0.009645,9.645463e-03,9.645463e-03,0.997611,-1.374964e-01,0.997611,-1.374964e-01,9.645463e-04
5,0.003000,0.001453,-0.003000,-1.452900e-03,1.109442e-12,0.000000e+00,0.004823,4.822733e-03,4.822733e-03,0.997611,-1.374964e-01,0.997611,-1.374964e-01,4.822733e-04
6,0.015000,0.007264,-0.015000,-7.264500e-03,2.773605e-11,0.000000e+00,0.024114,2.411366e-02,2.411366e-02,0.997611,-1.374965e-01,0.997611,-1.374965e-01,2.411366e-03
7,0.012000,0.005812,-0.012000,-5.811600e-03,1.775106e-11,0.000000e+00,0.019291,1.929092e-02,1.929092e-02,0.997611,-1.374965e-01,0.997611,-1.374964e-01,1.929092e-03
8,0.009000,0.004359,-0.009000,-4.358700e-03,9.984976e-12,8.673617e-19,0.014468,1.446819e-02,1.446819e-02,0.997611,-1.374964e-01,0.997611,-1.374964e-01,1.446819e-03
9,0.006000,0.002906,-0.006000,-2.905800e-03,4.437766e-12,0.000000e+00,0.009645,9.645463e-03,9.645463e-03,0.997611,-1.374964e-01,0.997611,-1.374964e-01,9.645463e-04
